In [4]:
# Step 1: Install Dependencies
!pip install transformers datasets torch

# Step 2: Load a Small Pre-installed Dataset (Wikitext-2)
from datasets import load_dataset

# Load Wikitext-2 dataset for training (raw version for better control)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Combine all text data from the dataset into one string
text_data = "\n".join(dataset["text"])

print(f"Sample Data:\n{text_data[:500]}")  # Display the first 500 characters

# Step 3: Preprocess the Dataset (Tokenization)
from transformers import GPT2Tokenizer

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Tokenize the entire text dataset
tokens = tokenizer(text_data, return_tensors="pt", max_length=512, truncation=True)

print(f"Tokenized Example:\n{tokens['input_ids'][:10]}")  # Display a tokenized sample

# Step 4: Load the GPT-2 Model
from transformers import GPT2LMHeadModel
import torch

# Load the pre-trained GPT-2 model with a language modeling head
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Move the model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Step 5: Train the Model (Fine-tuning on Wikitext-2)
from torch.optim import AdamW

# Prepare input data for training
input_ids = tokens["input_ids"].to(device)
attention_mask = tokens["attention_mask"].to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop: Fine-tune the model for a small number of epochs
epochs = 3  # Can be increased for better results
model.train()

for epoch in range(epochs):
    optimizer.zero_grad()  # Clear gradients from the previous step
    outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
    loss = outputs.loss  # Calculate the loss
    print(f"Epoch {epoch + 1} | Loss: {loss.item()}")

    loss.backward()  # Backpropagate the loss
    optimizer.step()  # Update model parameters

print("\nTraining Complete!")

# Step 6: Generate Text Based on User Input (Interactive Mode)
model.eval()

# Prompt the user to enter a text prompt
user_prompt = input("Enter a prompt to generate text: ")

# Tokenize the user input prompt
input_ids = tokenizer(user_prompt, return_tensors="pt").input_ids.to(device)

# Generate text using the fine-tuned model
output = model.generate(
    input_ids,
    max_length=100,  # Generate up to 100 tokens
    num_return_sequences=1,
    no_repeat_ngram_size=2,  # Avoid repetition
    early_stopping=True
)

# Decode and display the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"\nGenerated Text:\n{generated_text}")

# Step 7: Save the Fine-tuned Model (Optional)
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

print("\nModel and tokenizer saved to './fine_tuned_gpt2'")


Sample Data:

 = Valkyria Chronicles III = 


 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs
Tokenized Example:
tensor([[  198,   796,   569, 18354,  7496, 17740,  6711,   796,   220,   628,
           198,  2311,    73, 13090,   645,   569, 18354,  7496,   513,  1058,
           791, 47398, 17740,   357,  4960,  1058, 10545,   230,    99,   161,
           254,   112,  5641, 44444,  9202, 25084, 24440, 12675, 11839,    18,
           837,  6578,   764,   569, 18354,  7496,   286,   262, 30193,   513,
          1267,   837,  8811,  6412,   284,   355,   569, 18354,  7496, 

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Generated Text:
police, and the police department's own internal affairs unit.

The investigation into the shooting began in January, when a man who had been shot in the head was found dead in his car. The man's family said he had suffered a gunshot wound to the chest and was taken to a hospital. Police said the man had a history of mental illness and had recently been diagnosed with schizophrenia.

Model and tokenizer saved to './fine_tuned_gpt2'
